# Инциденты СТП

In [107]:
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials
import gspread
from datetime import datetime
import sqlalchemy
import pymysql
import warnings
import re
warnings.filterwarnings("ignore")

In [108]:
DF15_based = pd.read_excel("/home/python3/based_data/stp_based.xlsx")
DF15 = pd.read_excel("/home/python3/data_from_bi/ГЗ_3_02 Выгрузка инцидентов.xlsx", skiprows=3)
DF15 = DF15_based.append(DF15).drop_duplicates(subset=['Код инцидента']).reset_index(drop=True)

## Длительность обработки инцидентов СТП

In [109]:
def separation(data):
    data = data.split('\n')
    return data 
def separation_2(data):
    data = data[1]
    data = data.replace('Время \"Направлено в группу\": ', '')
    return data
def separation_3(data):
    data = data[3]
    data = data.replace('Время \"Выполнено\": ', '')
    return data
def separation_4(data):
    data = data[4]
    data = data.replace('Крайний срок: ', '')
    return data
def separation_5(data):
    data = data.split('_')
    data = data[0] 
    return data

In [110]:
DF15['Даты_2'] = DF15['Даты'].apply(separation)
DF15['time_directed_to_group'] = DF15['Даты_2'].apply(separation_2)
DF15['time_completed'] = DF15['Даты_2'].apply(separation_3)
DF15['deadline'] = DF15['Даты_2'].apply(separation_4)
DF15['is_completed'] = DF15['time_completed'] != ''

In [111]:
DF15['time_directed_to_group'] = pd.to_datetime(DF15[DF15['time_directed_to_group'] !='']['time_directed_to_group'], format = '%d.%m.%Y %H:%M:%S')
DF15['time_completed'] = pd.to_datetime(DF15[DF15['time_completed'] !='']['time_completed'], format = '%d.%m.%Y %H:%M:%S')
DF15['deadline'] = pd.to_datetime(DF15[DF15['deadline'] !='']['deadline'], format = '%d.%m.%Y %H:%M:%S')

In [112]:
DF15['duration'] = DF15['time_completed'] - DF15['time_directed_to_group']
DF15['delayed'] = DF15['deadline'] - DF15['time_directed_to_group']
DF15.loc[DF15['delayed'] > '0 days 00:00:00', 'delayed'] = ''

In [113]:
DF15['Исполнитель'] = DF15['Исполнитель'].astype(str).apply(separation_5)
DF15.loc[DF15['Исполнитель'] == 'nan', 'Исполнитель'] = ''

DF15['duration'] = DF15['duration'].dt.total_seconds()
DF15['delayed'] = abs(DF15['delayed']).dt.total_seconds()

In [114]:
stp = DF15.copy()
stp['Дата'] = stp['Даты'].apply(lambda x: pd.to_datetime(x[26:45], format="%d.%m.%Y %H:%M:%S"))
stp['Тип работ'] = stp['Тип работ'].fillna('0')
stp['session_id'] = stp['Тип работ'].apply(lambda x:
                    max([int(e) for e in re.split("[^0-9]", x) if e != ''], default=0))

# Убрать при переносе на основной код
DF5 = pd.read_excel("/home/python3/data_from_bi/ГЗ_1_07 Заказчики.xlsx", skiprows=2)          
DF9 = pd.read_excel("/home/python3/data_from_bi/ГЗ_2_01 Выгрузка item.xlsx", skiprows=2)      
DF11 = pd.read_excel("/home/python3/data_from_bi/ГЗ_2_03 Выгрузка sessions.xlsx", skiprows=2) 
DF11 = DF11.drop(columns='Unnamed: 6')                                                        

DF13 = pd.read_csv("/home/python3/data_from_bi/ГЗ_2_05 Выгрузка sessions_closed по дате обновления.csv")
DF13.columns = ['id', 'task id', 'status', 'verify code', 'created at', 'user id', 'updated at'] 
ints = ['id', 'task id', 'status', 'verify code', 'user id']
dates = ['created at', 'updated at'] 
for c in ints:
    DF13[c] = DF13[c].astype(int)
for c in dates:
    DF13[c] = pd.to_datetime(DF13[c], format='%d.%m.%Y %H:%M:%S')
DF13_based = pd.read_csv("/home/python3/based_data/sessions_closed_based.csv")
ints = ['id', 'task id', 'status', 'verify code', 'user id']
dates = ['created at', 'updated at'] 
for c in ints:
    DF13_based[c] = DF13_based[c].astype(int)
for c in dates:
    DF13_based[c] = pd.to_datetime(DF13_based[c], format='%Y-%m-%d %H:%M:%S')
DF13 = DF13_based.append(DF13).drop_duplicates(subset=['id'], keep='last')
# ----------------------------------------------------------------------------------------------------------------------------
df11 = DF11.copy()
df13 = DF11.copy()
df13 = DF13.append(df11)
df9 = DF9.copy()
df5 = DF5.copy()

df13 = df13[['id', 'task id']]
df13.columns = ['session_id', 'task_id']

df9 = df9[['id', 'name', 'customer id']]
df9.columns = ['task_id', 'name', 'customer_id']

df5 = df5[['id', 'Название заказчика']]
df5.columns = ['customer_id', 'Название заказчика']

stp = stp.merge(df13, how='left', on='session_id')
stp = stp.merge(df9, how='left', on='task_id')
stp = stp.merge(df5, how='left', on='customer_id')

def set_quartal(month):
    q = {
    1: 'Q1',
    2: 'Q2',
    3: 'Q3',
    4: 'Q4'}
    return q[month//4 + 1]

stp['month'] = [str(row.date())[:7] for row in stp['Дата']]
stp['quartal'] = [f"{str(row.year)}-{set_quartal(row.month)}" for row in stp['Дата']]
stp['День'] = [str(row.date())[:11] for row in stp['Дата']]


In [115]:
stp = stp[['Код инцидента', 'Группа услуг', 'Категория', 'услуга', 
             'Сервис','Исполнитель','time_directed_to_group', 
             'time_completed', 'deadline', 'is_completed', 'duration', 
             'delayed', 'Дата', 'session_id', 'task_id', 'name','customer_id', 
             'Название заказчика', 'month', 'quartal', 'День']]
stp.columns = ['incident_code', 'service_group', 'сategory', 'service_1', 
                'service_2', 'responsible_person', 'time_directed_to_group',
                'time_completed', 'deadline', 'is_completed',	'duration',	'delayed', 
                'date', 'session_id', 'task_id', 'name','customer_id', 
                'customеr_name', 'month', 'quartal', 'day']
stp.to_excel("/home/python3/Bashir/stp_based.xlsx", index=False)

In [116]:
#connections settings
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
wks_name = 'main'
credentials = ServiceAccountCredentials.from_json_keyfile_name('/home/python3/cron_scripts_and_keys/.....json', scope)
gc = gspread.authorize(credentials)
cnf = {
  'user': '',
  'password': '',
  'host': '',
  'database': ''
}
connection = sqlalchemy.create_engine(f"mysql+pymysql://{cnf['user']}:{cnf['password']}@{cnf['host']}/{cnf['database']}")

In [117]:
stp.to_sql(con=connection, name='stp_burndown', if_exists='replace')

9854